In [ ]:
import torch
import mne
import numpy
from gnn import GraphLearner
from chbmit import chbmit
from torch_geometric.utils.convert import to_networkx

In [ ]:
patient_0 = chbmit.CHB_MIT_PAITENT(tok_len=60, ctx_len=1, path="")

In [ ]:
network = to_networkx(data=data)
kx.draw(network)